In [1]:
import pandas as pd

In [2]:
dataset = pd.read_json("public-dataset.json")

In [3]:
df = pd.DataFrame(dataset)

In [4]:
print(dataset)

       buynow_price rodzaj karty graficznej  \
70           1599.0    grafika zintegrowana   
1489         2999.0      grafika dedykowana   
407          2999.0    grafika zintegrowana   
1359         2849.0    grafika zintegrowana   
10567        2319.0                    None   
...             ...                     ...   
10541        1329.0    grafika zintegrowana   
8281          979.0                    None   
8053         2565.0      grafika dedykowana   
8727         4079.0      grafika dedykowana   
5978         2949.0      grafika dedykowana   

                                             komunikacja  \
70              [wi-fi, bluetooth, lan 10/100/1000 mbps]   
1489               [nfc (near field communication), gps]   
407             [wi-fi, bluetooth, lan 10/100/1000 mbps]   
1359            [wi-fi, bluetooth, lan 10/100/1000 mbps]   
10567                                               None   
...                                                  ...   
10541          

In [5]:
df = df.drop("stan", axis = 1) #druga opcja bez zapisu nowego df df.drop("stan", inplace = True)

In [6]:
df.dtypes

buynow_price                         float64
rodzaj karty graficznej               object
komunikacja                           object
rozdzielczość (piksele)               object
liczba rdzeni procesora               object
wielkość pamięci ram                  object
system operacyjny                     object
typ dysku twardego                    object
sterowanie                            object
multimedia                            object
typ pamięci ram                       object
taktowanie bazowe procesora (ghz)    float64
seria procesora                       object
pojemność dysku (gb)                 float64
gwarancja                             object
wielkość matrycy                      object
dtype: object

In [7]:
df["komunikacja"].apply(pd.Series).stack().unique().tolist()

['wi-fi',
 'bluetooth',
 'lan 10/100/1000 mbps',
 'nfc (near field communication)',
 'gps',
 'lan 10/100 mbps',
 'intel wireless display (widi)',
 'modem 3g (wwan)',
 'wi-fi 802.11 a/b/g/n/ac',
 'wi-fi 802.11 b/g/n/ac',
 'modem 4g (lte)',
 'wi-fi 802.11 b/g/n',
 'wi-fi 802.11 a/b/g/n']

In [8]:
df["bluetooth"] = (df["komunikacja"].str.contains('bluetooth', regex=False)*1)

In [9]:
(df["komunikacja"].str.contains('bluetooth', regex=False)*1).sum()

6524

In [10]:
df["wi-fi"] = (df["komunikacja"].str.contains('wi-fi', regex=False)*1)

In [11]:
df["lan 10/100 mbps"] = (df["komunikacja"].str.contains('lan 10/100 mbps', regex=False)*1).sum()

In [12]:
df["lan 10/100/1000 mbps"] = (df["komunikacja"].str.contains('lan 10/100/1000 mbps', regex=False)*1).sum()

In [13]:
(df["komunikacja"].str.contains('nfc (near field communication)', regex=False)*1).sum()

1454

In [14]:
(df["komunikacja"].str.contains('modem 3g (wwan)', regex=False)*1).sum()

1009

In [15]:
(df["komunikacja"].str.contains('gps', regex=False)*1).sum()

269

In [16]:
(df["komunikacja"].str.contains('intel wireless display (widi)', regex=False)*1).sum()

1004

In [17]:
(df["komunikacja"].str.contains('modem 4g (lte)', regex=False)*1).sum()

267

In [18]:
(df["komunikacja"].str.contains('wi-fi 802.11 b/g/n', regex=False)*1).sum()

73

In [19]:
(df["komunikacja"].str.contains('wi-fi 802.11 b/g/n/ac', regex=False)*1).sum()

166

In [20]:
(df["komunikacja"].str.contains('wi-fi 802.11 a/b/g/n/ac', regex=False)*1).sum()

143

In [21]:
(df["komunikacja"].str.contains('wi-fi 802.11 a/b/g/n', regex=False)*1).sum()

17

In [22]:
df.shape

(7853, 20)

In [23]:
160/7853

0.020374379218133197